In [1]:
import tensorflow as tf
import numpy as np
import datetime
import time

import sklearn.metrics

In [76]:
def genObs(length):
    return np.random.choice(range(3), size=length)

def genTarget(x):
    return ''.join([str(z) for z in x]).count('012') - ''.join([str(z) for z in x]).count('210')

def genSample(num, length=20):
    x = [genObs(length=length) for _ in range(num)]
    y = [genTarget(t) for t in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

In [77]:
train_x, train_y = genSample(10000)
valid_x, valid_y = genSample(1000)
valid_x[:10], valid_y[:10]

(array([[0, 1, 0, 0, 1, 2, 2, 1, 1, 0, 1, 2, 0, 0, 1, 2, 1, 0, 2, 1],
        [1, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 2, 1, 2],
        [2, 2, 2, 1, 0, 2, 1, 0, 1, 2, 0, 1, 1, 1, 2, 2, 2, 1, 0, 2],
        [2, 0, 2, 2, 0, 1, 2, 1, 2, 1, 1, 0, 2, 0, 0, 1, 1, 1, 0, 1],
        [1, 1, 0, 2, 1, 2, 2, 0, 1, 0, 0, 1, 1, 1, 2, 1, 0, 2, 2, 0],
        [1, 2, 0, 2, 2, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1],
        [1, 0, 2, 1, 2, 2, 1, 2, 0, 2, 2, 1, 2, 0, 2, 0, 2, 1, 1, 1],
        [1, 0, 1, 1, 0, 1, 0, 0, 2, 1, 1, 1, 0, 0, 0, 2, 1, 2, 2, 0],
        [0, 1, 1, 0, 2, 1, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 0, 2, 1, 1],
        [2, 2, 1, 0, 1, 0, 2, 0, 2, 2, 0, 1, 0, 1, 1, 2, 2, 1, 1, 2]]),
 array([ 2, -2, -2,  1, -1,  0,  0,  0, -1, -1]))

In [78]:
SEQ_LEN = 20
RNN_SIZE = [10]

InnerCell = lambda n: tf.nn.rnn_cell.BasicRNNCell(num_units=n, activation=tf.nn.relu)
#InnerCell = lambda n: tf.nn.rnn_cell.LSTMCell(num_units=n, activation=tf.nn.relu, use_peepholes=True)

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.float32)
tfi_y = tf.placeholder(shape=(None), dtype=tf.float32)

tfX = tf.reshape(tfi_x, shape=(tf.shape(tfi_x)[0], tf.shape(tfi_x)[1], 1))
tfY = tf.reshape(tfi_y, shape=(tf.shape(tfi_y)[0], 1))


rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE])
#rnnCell = tf.nn.rnn_cell.GRUCell(RNN_SIZE[0])

_, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

tfOut = tf.layers.dense(tfO[-1], 1)

tfLoss = tf.sqrt(tf.reduce_mean(tf.square(tfY - tfOut)))
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

#tfOutP = tf.nn.softmax(tfOut)
tfOutR = tf.cast(tf.round(tfOut),dtype=tf.int64)

tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tf.cast(tfY, dtype=tf.int64), tf.cast(tf.round(tfOut),dtype=tf.int64)), dtype=tf.float32))
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [79]:
batch_size = 1000
num_steps  = 30
num_epochs = 500

fmtstr = 'Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.3f}\t\tAC:{5:1.3f}'
valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini_x, mini_y = randomBatch((train_x, train_y), batchSize=batch_size)
        train_batch = {tfi_x:mini_x, tfi_y:mini_y}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv = tfLoss.eval(feed_dict=valid_batch)
        ac = tfAccuracy.eval(feed_dict=valid_batch)
        if i%10 == 0:
            print(fmtstr.format(i,t1-t0,l0,l1,lv,ac))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (0.203 sec): loss changed from 1.04 to 1.01		VL:1.049		AC:0.386
Epoch 10 (0.183 sec): loss changed from 0.717 to 0.646		VL:0.680		AC:0.535
Epoch 20 (0.182 sec): loss changed from 0.441 to 0.426		VL:0.454		AC:0.736
Epoch 30 (0.195 sec): loss changed from 0.346 to 0.331		VL:0.356		AC:0.847
Epoch 40 (0.184 sec): loss changed from 0.279 to 0.276		VL:0.300		AC:0.901
Epoch 50 (0.184 sec): loss changed from 0.246 to 0.239		VL:0.271		AC:0.922
Epoch 60 (0.23 sec): loss changed from 0.265 to 0.242		VL:0.250		AC:0.938
Epoch 70 (0.187 sec): loss changed from 0.209 to 0.222		VL:0.245		AC:0.953
Epoch 80 (0.216 sec): loss changed from 0.196 to 0.189		VL:0.210		AC:0.966
Epoch 90 (0.183 sec): loss changed from 0.181 to 0.178		VL:0.189		AC:0.985
Epoch 100 (0.182 sec): loss changed from 0.164 to 0.172		VL:0.187		AC:0.990
Epoch 110 (0.183 sec): loss changed from 0.164 to 0.154		VL:0.165		AC:0.997
Epoch 120 (0.183 sec): loss changed from 0.147 to 0.149		VL:0.164		AC:0.997
Epoch 130 (0.183 sec): los

In [80]:
valid_r[:10]

array([[ 2],
       [-2],
       [-2],
       [ 1],
       [-1],
       [ 0],
       [ 0],
       [ 0],
       [-1],
       [-1]], dtype=int64)

In [81]:
valid_y[:10]

array([ 2, -2, -2,  1, -1,  0,  0,  0, -1, -1])